# Emotion analysis

In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import nltk
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Modelling
from sklearn.model_selection import train_test_split,KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.svm import SVC

In [7]:


import requests
from bs4 import BeautifulSoup

# Replace with the URL of your Facebook group
group_url = "https://www.facebook.com/DutConfessions/"

# Send a GET request to the group's URL and get the HTML content
response = requests.get(group_url)
html_content = response.content

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Extract the URLs of the posts in the group
post_urls = []
for link in soup.find_all("a"):
    href = link.get("href")
    if href and "/permalink/" in href:
        post_urls.append(href)

# For each post URL, extract the comments using BeautifulSoup
comments = []
for post_url in post_urls:
    # Send a GET request to the post URL and get the HTML content
    response = requests.get(post_url)
    html_content = response.content

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")

    # Extract the comments from the HTML content
    for comment in soup.find_all("div", {"class": "xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs"}):
        text = comment.get_text(strip=True)
        comments.append(text)

# Print the collected comments
print(comments)

[]


In [2]:
nltk.download('omw-1.4')
nltk.download("stopwords")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\huuhu\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\huuhu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Read dataset 

In [3]:
df_train = pd.read_csv('D:/Learning/Data_Analytics/Crawl Data/End_Term_EN/DataSet/test.txt', names=['Text', 'Emotion'], sep=';')
df_val = pd.read_csv('D:/Learning/Data_Analytics/Crawl Data/End_Term_EN/DataSet/val.txt', names=['Text', 'Emotion'], sep=';')
df_test = pd.read_csv('D:/Learning/Data_Analytics/Crawl Data/End_Term_EN/DataSet/test.txt', names=['Text', 'Emotion'], sep=';')

In [4]:
df_train.head()

,Text,Emotion
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


In [5]:
df_train.Emotion.value_counts()

joy         695
sadness     581
anger       275
fear        224
love        159
surprise     66
Name: Emotion, dtype: int64

In [6]:
df_val.head()

,Text,Emotion
0,im feeling quite sad and sorry for myself but ...,sadness
1,i feel like i am still looking at a blank canv...,sadness
2,i feel like a faithful servant,love
3,i am just feeling cranky and blue,anger
4,i can have for a treat or if i am feeling festive,joy


In [7]:
df_test.head()

,Text,Emotion
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


## Processing 

### Check duplicate
- There is just one row which is duplicated but we need to check also if there are rows having the same text but different emotions

In [8]:
df_train.duplicated().sum()

0

In [28]:
df_test.duplicated().sum()

1

In [29]:
df_val.duplicated().sum()

1

In [9]:
def remove_duplicate_row(df):
    index = df[df.duplicated() == True].index
    df.drop(index, axis = 0, inplace = True)
    df.reset_index(inplace=True, drop = True)

In [10]:
df_train['Text'].duplicated().sum()

0

In [11]:
def remove_diplicate_dif_emotion(df):
    index = df[df['Text'].duplicated() == True].index
    df.drop(index, axis = 0, inplace = True)
    df.reset_index(inplace=True, drop = True)

## Cleaning

In [12]:
stop_words = set(stopwords.words("english"))

In [13]:
def lemmatization(text):
    lemmatizer= WordNetLemmatizer()
    text = text.split()
    text=[lemmatizer.lemmatize(y) for y in text]
    return " " .join(text)

In [14]:
def remove_stop_words(text):

    Text=[i for i in str(text).split() if i not in stop_words]
    return " ".join(Text)

In [15]:
def Removing_numbers(text):
    text=''.join([i for i in text if not i.isdigit()])
    return text

In [16]:
def lower_case(text):
    
    text = text.split()

    text=[y.lower() for y in text]
    
    return " " .join(text)

In [17]:
def Removing_punctuations(text):
    ## Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = text.replace('؛',"", )
    
    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text =  " ".join(text.split())
    return text.strip()

In [18]:
def Removing_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [19]:
def remove_small_sentences(df):
    for i in range(len(df)):
        if len(df.text.iloc[i].split()) < 3:
            df.text.iloc[i] = np.nan

In [20]:
def normalize_text(df):
    df.Text=df.Text.apply(lambda text : lower_case(text))
    df.Text=df.Text.apply(lambda text : remove_stop_words(text))
    df.Text=df.Text.apply(lambda text : Removing_numbers(text))
    df.Text=df.Text.apply(lambda text : Removing_punctuations(text))
    df.Text=df.Text.apply(lambda text : Removing_urls(text))
    df.Text=df.Text.apply(lambda text : lemmatization(text))
    return df

In [21]:
def normalized_sentence(sentence):
    sentence= lower_case(sentence)
    sentence= remove_stop_words(sentence)
    sentence= Removing_numbers(sentence)
    sentence= Removing_punctuations(sentence)
    sentence= Removing_urls(sentence)
    sentence= lemmatization(sentence)
    return sentence

In [22]:
df_train= normalize_text(df_train)
df_test= normalize_text(df_test)
df_val= normalize_text(df_val)

## Modeling

### Creating a pipline using tf-idf for words embedding and different models.

In [23]:
#Preprocess text
X_train = df_train['Text'].values
y_train = df_train['Emotion'].values

X_test = df_test['Text'].values
y_test = df_test['Emotion'].values

X_val = df_val['Text'].values
y_val = df_val['Emotion'].values


In [24]:
def train_model(model, data, targets):
    """
    Train a model on the given data and targets.
    
    Parameters:
    model (sklearn model): The model to be trained.
    data (list of str): The input data.
    targets (list of str): The targets.
    
    Returns:
    Pipeline: The trained model as a Pipeline object.
    """
    # Create a Pipeline object with a TfidfVectorizer and the given model
    text_clf = Pipeline([('vect',TfidfVectorizer()),
                         ('clf', model)])
    # Fit the model on the data and targets
    text_clf.fit(data, targets)
    return text_clf

In [25]:
def get_F1(trained_model,X,y):

    predicted=trained_model.predict(X)
    # Calculate the F1 score for the predictions
    f1=f1_score(y,predicted, average=None)
    # Return the F1 score
    return f1

In [26]:
#Train the model with the training data
log_reg = train_model(RandomForestClassifier(random_state = 0), X_train, y_train)

In [27]:
csv_path = 'D:/Learning/Data_Analytics/Crawl Data/End_Term/output.csv'
data = pd.read_csv(csv_path)
data

FileNotFoundError: [Errno 2] No such file or directory: 'D:/Learning/Data_Analytics/Crawl Data/End_Term/output.csv'

In [ ]:
#Make a single prediction
y_pred=log_reg.predict(['love'])
y_pred

array(['joy'], dtype=object)

In [ ]:
xxxx =log_reg.predict([ 'my old brother is dead'])
print(xxxx)

['fear']


In [ ]:
#test the model with the test data
y_pred=log_reg.predict(X_test)
print(y_pred)
#calculate the accuracy
log_reg_accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ', log_reg_accuracy,'\n')

#calculate the F1 score
f1_Score = get_F1(log_reg,X_test,y_test)
pd.DataFrame(f1_Score, index=df_train.Emotion.unique(), columns=['F1 score'])

['sadness' 'sadness' 'sadness' ... 'joy' 'joy' 'fear']
Accuracy:  0.8905 



,F1 score
sadness,0.901460
anger,0.872807
love,0.909601
surprise,0.748299
fear,0.934726
joy,0.619048


# Craw data and predict emotion

In [ ]:
def crawl_sentence(word_search):

    url = 'https://sentence.yourdictionary.com/'

    source_code = requests.get(url + word_search).text
    soup = BeautifulSoup(source_code, 'html.parser')

    content_list = []

    for each_text in soup.findAll('p', {'class': 'sentence-item__text'}):
        content = each_text.text
        content_list.append(content)

    # Create a Pandas dataframe from the data.
    df = pd.DataFrame({'Sentence': content_list})
    df.to_csv(f'{word_search}''.csv', index=False)

In [ ]:
crawl_sentence("love")

In [ ]:
def load_data_excel(file_path):
    data_frame = pd.read_excel(file_path, usecols='B')
    return data_frame['Data']

In [ ]:
df_train

,Text,Emotion
0,didnt feel humiliated,sadness
1,go feeling hopeless damned hopeful around some...,sadness
2,im grabbing minute post feel greedy wrong,anger
3,ever feeling nostalgic fireplace know still pr...,love
4,feeling grouchy,anger
...,...,...
15995,brief time beanbag said anna feel like beaten,sadness
15996,turning feel pathetic still waiting table subb...,sadness
15997,feel strong good overall,joy
15998,feel like rude comment im glad,anger
